In [ ]:
# This script trains the model cf_re_ds
# Augmentation: Individual images are randomly augmented, not at epoch end

import sys
#adjust your path correspondingly
sys.path.insert(0,'/environment/workdir/Documents/uhaworkspace/kitti/training_cf/model_arch/libs')

import numpy as np
import os
from tensorflow import keras
from datetime import datetime, timedelta
from utils import train_val_split, split_check, write_path, kittiroad
from ds_layer_p2p import DS1_activate 
####################################################################################

In [ ]:
import tensorflow as tf
# ====================================dir_name = os.path.join(model_dir,"model_lite_cf_ds")
# model=keras.models.load_model(dir_name,custom_objects={'DS1_activate': DS1_activate})
# Required only on the local gpu rtx 2070 (may be due to some memory issue)

physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Memory settings
from tensorflow.compat.v1.keras.backend import set_session
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True # dynamically grow the memory used on the GPU
config.log_device_placement = True # to log device placement (on which device the operation ran)
sess = tf.compat.v1.Session(config=config)
set_session(sess)
## ---- end Memory setting ----

#============================================

2023-01-18 20:47:08.688119: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:08.692190: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:08.692521: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:08.837123: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6



2023-01-18 20:47:09.147385: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:09.147877: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:09.148246: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-18 20:47:09.148584: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5323 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [1]:
#paths to be updated
input_dir_rgb="/environment/workdir/Documents/uhaworkspace/kitti/training_cf/image_2/"
input_dir_velo="/environment/workdir/Documents/uhaworkspace/kitti/training_cf/lidar/"
target_dir="/environment/workdir/Documents/uhaworkspace/kitti/training_cf/semantic_rgb/"

out_dir= "/environment/workdir/Documents/uhaworkspace/kitti/training_cf/output/"
model_dir= "/environment/workdir/Documents/uhaworkspace/kitti/training_cf/model_arch/"
weights_dir =  "/environment/workdir/Documents/uhaworkspace/kitti/training_cf/backp_weights_rd/"

In [ ]:
##zero weights
# # Load model arch
# dir_name = os.path.join(model_dir,"model_lite_cf_ds")
# model=keras.models.load_model(dir_name,custom_objects={'DS1_activate': DS1_activate})

## RD weightsThe dataset does not provide LiDAR images. Therefore,  corespondent 3d point-cloud data for the camera images is driven to be discovered in the raw dataset \cite{Geiger2013IJRR}. Thus, 127 camera images are identified, and their corresponding LiDAR frames. The latter ones are projected, up-sampled to dense depth images \cite{geletu2022deep}, \cite{premebida2014pedestrian} and fused to the CF-Evi model together with the camera images, like in the road detection case. Since the CF-Evi model is developed for road segmentation and aims for low complexity, the number of classes is simplified for 3 objects: road, car and background, where background represents everything else than road and cars.
dir_name = os.path.join(model_dir,"model_lite_cf_ds")
model=keras.models.load_model(dir_name,custom_objects={'DS1_activate': DS1_activate})
model.load_weights(os.path.join(weights_dir,"checkpoint_cross_fusion"))
model.summary()

######################################################################################
# input size
img_size = (384,1248)
batch_size = 1

#Prepare paths of input images and target segmentation masks   
data_split= train_val_split( input_dir_rgb, input_dir_velo, target_dir)
train_input_img_paths_rgb= data_split["train_cam_1"]  # path of camera images, training
train_input_img_paths_velo= data_split["train_velo_1"] # path of velodyne projected, training
train_target_img_paths= data_split["train_target_1"]     # path of target image, training
val_input_img_paths_rgb= data_split["val_cam_1"]    # path of camera images, validation
val_input_img_paths_velo= data_split["val_velo_1"]   # path of velodyne projected, validation
val_target_img_paths= data_split["val_target_1"]       # path of taget image, validation 

# prepare split matching
train_cam= split_check(train_input_img_paths_rgb)
train_velo= split_check(train_input_img_paths_velo)
train_target= split_check(train_target_img_paths)
val_cam= split_check(val_input_img_paths_rgb)
val_velo= split_check(val_input_img_paths_velo)
val_target= split_check(val_target_img_paths)

assert train_cam == train_velo and train_velo == train_target, 'INVALID SPLIT: training splits not matching'
assert val_cam == val_velo and val_velo == val_target, 'INVALID SPLIT: validation splits not matching'
assert not any(item in train_cam for item in val_cam), 'INVALID SPLIT: validation split contains training split'

# write split paths to a text file
#paths to be updated
write_path(out_dir, train_input_img_paths_rgb, 'train_input_img_paths_rgb.txt')
write_path(out_dir, train_input_img_paths_velo, 'train_input_img_paths_velo.txt')
write_path(out_dir, train_target_img_paths, 'train_target_img_paths.txt')

write_path(out_dir, val_input_img_paths_rgb, 'val_input_img_paths_rgb.txt')
write_path(out_dir, val_input_img_paths_velo, 'val_input_img_paths_velo.txt')
write_path(out_dir, val_target_img_paths, 'val_target_img_paths.txt')

# Instantiate data Sequences for each split
train_gen = kittiroad(batch_size, img_size, train_input_img_paths_rgb,train_input_img_paths_velo, train_target_img_paths)
val_gen = kittiroad(batch_size, img_size, val_input_img_paths_rgb,val_input_img_paths_velo, val_target_img_paths,val=True)

# Configure the model for training.
# Polynomial Decay  

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 lidar (InputLayer)             [(None, 384, 1248,   0           []                               
                                3)]                                                               
                                                                                                  
 rgb (InputLayer)               [(None, 384, 1248,   0           []                               
                                3)]                                                               
                                                                                                  
 Block1_lidar_zp (ZeroPadding2D  (None, 386, 1250, 3  0          ['lidar[0][0]']                  
 )                              )                                                             

In [ ]:

starter_learning_rate=0.0005
end_learning_rate=0
epochs =500
decay_steps=261*epochs # 261(frames)x200(epoch) 
lr_schedule=keras.optimizers.schedules.PolynomialDecay(starter_learning_rate, decay_steps,end_learning_rate,power=0.9)

# Optimizer
opt=keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile
model.compile(optimizer=opt, loss= keras.losses.MeanSquaredError(), metrics= ["mse"])
#model.compile(optimizer=opt, metrics= ["mse"])

# Modelcheckpoint
model_checkpoint_callback=keras.callbacks.ModelCheckpoint(
filepath= os.path.join(out_dir,'checkpoint_cross_fusion'),
save_weights_only=True,
monitor='val_loss', 
mode='min',
save_best_only=True)

# Train the model

st_time= datetime.now()
history= model.fit(train_gen, epochs=epochs, validation_data=val_gen, callbacks=[model_checkpoint_callback])
end_time= datetime.now()

# Save training miscellaneous results
elapsed_total_sec= (end_time - st_time).total_seconds()
conversion = timedelta(seconds= elapsed_total_sec)

with open(os.path.join(out_dir, 'time_log.txt'), 'w') as f:
    f.write('%s\n' %'Training time in H:M:S')
    f.write( str(conversion))
    f.close()

np.savez_compressed(os.path.join(out_dir,'history.npz'), loss= history.history['loss'], 
                val_loss= history.history['val_loss'], 
               accuracy= history.history['mse'],
               val_accuracy= history.history['val_mse'] )





Epoch 1/500


2023-01-18 20:52:15.847040: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8101
2023-01-18 20:52:16.329418: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-18 20:52:16.329758: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-18 20:52:16.329766: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Couldn't invoke ptxas --version
2023-01-18 20:52:16.330153: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2023-01-18 20:52:16.330209: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


114/114 [==============================] - 19s 125ms/step - loss: 0.1770 - mse: 0.1482 - val_loss: 0.1861 - val_mse: 0.1424
Epoch 2/500
114/114 [==============================] - 14s 125ms/step - loss: 0.1783 - mse: 0.1469 - val_loss: 0.1853 - val_mse: 0.1413
Epoch 3/500
114/114 [==============================] - 14s 126ms/step - loss: 0.1778 - mse: 0.1473 - val_loss: 0.1850 - val_mse: 0.1410
Epoch 4/500
114/114 [==============================] - 15s 129ms/step - loss: 0.1782 - mse: 0.1479 - val_loss: 0.1848 - val_mse: 0.1407
Epoch 5/500
114/114 [==============================] - 15s 129ms/step - loss: 0.1779 - mse: 0.1474 - val_loss: 0.1847 - val_mse: 0.1409
Epoch 6/500
114/114 [==============================] - 15s 130ms/step - loss: 0.1765 - mse: 0.1477 - val_loss: 0.1845 - val_mse: 0.1407
Epoch 7/500
114/114 [==============================] - 15s 131ms/step - loss: 0.1764 - mse: 0.1471 - val_loss: 0.1843 - val_mse: 0.1407
Epoch 8/500
114/114 [==============================] - 15s 1